In [8]:
from pathlib import Path
import os

# Ollama server (local)
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "http://localhost:11434")
MODEL_NAME = "llama3.2:3b"

# go one level up from notebook folder
BASE_DIR = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent

DATA_DIR = (BASE_DIR / "data" / "sample").resolve()
DB_DIR = (BASE_DIR / "data" / "chroma_db").resolve()
DB_DIR.mkdir(parents=True, exist_ok=True)

print("OLLAMA_HOST:", OLLAMA_HOST)
print("MODEL:", MODEL_NAME)
print("DATA_DIR:", DATA_DIR)
print("DB_DIR:", DB_DIR)


OLLAMA_HOST: http://localhost:11434
MODEL: llama3.2:3b
DATA_DIR: D:\+Job\rag-job-assistant\data\sample
DB_DIR: D:\+Job\rag-job-assistant\data\chroma_db


In [9]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
import torch

# 1️⃣ Load documents
docs = []
for p in DATA_DIR.glob("*"):
    if p.suffix.lower() == ".pdf":
        docs += PyPDFLoader(str(p)).load()
    elif p.suffix.lower() in {".txt", ".md"}:
        docs += TextLoader(str(p), encoding="utf-8").load()

print(f"Loaded {len(docs)} documents")

# 2️⃣ Chunk them
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""],
)
chunks = splitter.split_documents(docs)
print(f"Chunked into {len(chunks)} chunks")

# 3️⃣ Embeddings (GPU if available, else CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
emb = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={"device": device},
)

# 4️⃣ Vector store (auto-persists in Chroma ≥0.4)
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=emb,
    persist_directory=str(DB_DIR),
)
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

print(f"Chroma built & persisted using device: {device}")


Loaded 1 documents
Chunked into 6 chunks


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Chroma built & persisted using device: cuda


In [10]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(
    base_url=OLLAMA_HOST,
    model=MODEL_NAME,
    temperature=0.2,
)

prompt = ChatPromptTemplate.from_template(
    "Use the context to answer the question. Be concise.\n\nContext:\n{context}\n\nQuestion: {question}"
)

def format_docs(docs):
    return "\n\n".join(f"[{i+1}] " + d.page_content for i,d in enumerate(docs))

def rag_answer(query: str):
    retrieved = retriever.invoke(query)
    ctx = format_docs(retrieved)
    chain = prompt | llm | StrOutputParser()
    resp = chain.invoke({"context": ctx, "question": query})
    return resp, retrieved

print("LLM & RAG chain ready.")


LLM & RAG chain ready.


In [11]:
resp, ctx_docs = rag_answer("Summarize the key points from these files.")
print(resp)


Here is a concise summary of the key points:

Sheikh Abdul Munim has experience in:

* Building real-time hazard detection and localization pipelines using camera, TF2, and RViz.
* Designing and testing electro-mechanical systems, including gear shifters and brake system plausibility devices.
* Leading coding courses and teaching programming languages like Python.
* Working on autonomous navigation systems and improving vehicle performance through data-driven optimization.

He has a strong background in:

* Artificial Intelligence (AI) and Machine Learning (ML)
* Mechatronics and robotics
* Programming languages like Python, MATLAB, and ROS1/ROS2

He holds degrees from RMIT University (Master of AI) and LPU (B.Tech. in Mechanical Engineering - Mechatronics).


In [7]:
for i, d in enumerate(ctx_docs, 1):
    print(f"\n[{i}] Source:", d.metadata.get("source"))
    print(d.page_content[:400], "...")



[1] Source: D:\+Job\rag-job-assistant\data\sample\munim_cv.pdf
Group Project Mar 2025 – Apr 2025
–Built real-time hazard detection and localisation pipeline using camera, TF2, and RViz.
–Enabled reliable hazard positioning and safe exploration triggers.
–Published system status updates to support autonomous navigation.
Formula Bharat - F6 Category Racing Car LPU
Electronic team lead May 2019 – Feb 2020
–Designed electro-mechanical gear shifter and Brake Syste ...

[2] Source: D:\+Job\rag-job-assistant\data\sample\munim_cv.pdf
Sheikh Abdul Munim
0481973546 |sheikh00munim@gmail.com |LinkedIn |/gtbGitHub |/g♀bePortfolio
Education
Royal Melbourne Institute of Technology (RMIT University) Victoria, Australia
Master of Artificial Intelligence March 2024 – Ongoing
Relevant Coursework: Programming Autonomous Robot, Intelligent Decision Making, Artificial Intelligence,
Computational Machine Learning
Lovely Professional Universi ...

[3] Source: D:\+Job\rag-job-assistant\data\sample\munim_cv.pd